In [59]:
import os
import time

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn

from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import CosineAnnealingLR

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns


In [60]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


TRAIN_AUG_CSV_PATH = '/root/data/train_kr_aug.csv'
TRAIN_AUG_IMAGE_PATH = '/root/data/train_aug'

VALID_CSV_PATH = '/root/data/valid.csv'
VALID_IMAGE_PATH = '/root/data/valid'

TEST_CSV_PATH = '/root/data/sample_submission.csv'
TEST_IMAGE_PATH = '/root/data/test'

RESULT_CSV_PATH = '/root/'


# HyperParameter

In [61]:
# training config
img_size = 380
LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 32
num_workers = 0

patience = 5
min_delta = 0.001 # 성능 개선의 최소 변화량

# 1. DATA LOAD

In [62]:
# test image 변환
data_transform = A.Compose([
    A.Resize(height = img_size, width = img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)['image']
    
        return img, target, name  # 파일 이름 반환

    def get_labels(self):
        return self.df[:, 1]

trn_dataset = ImageDataset(
    TRAIN_AUG_CSV_PATH,
    TRAIN_AUG_IMAGE_PATH,
    transform = data_transform
)

val_dataset = ImageDataset(
    VALID_CSV_PATH,
    VALID_IMAGE_PATH,
    transform = data_transform
)

tst_dataset = ImageDataset(
    TEST_CSV_PATH,
    TEST_IMAGE_PATH,
    transform = data_transform
)

labels = trn_dataset.get_labels()
labels = labels.astype(int)

# DataLoader
trn_loader = DataLoader(
    trn_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = num_workers,
    pin_memory = True,
    drop_last = False
)

val_loader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 0,
    pin_memory = True,
    drop_last = False
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = 0,
    pin_memory = True
)

print(len(trn_dataset), len(tst_dataset))

25120 3140


# 2. Model Train

In [63]:
# 모델 생성 및 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('efficientnet_b0',
                        pretrained=True,
                        num_classes = 17).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = LR)


In [64]:
# 한 epoch 학습 함수
def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for step, (image, targets, _) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "model": model,
        "train_epoch": epoch,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1
    }

    return ret


In [65]:
# 한 epoch 검증 함수
def valid_one_epoch(loader, model, loss_fn, device, epoch):
    model.eval()
    valid_loss = 0

    preds_list = []
    targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader)
        for step, (image, targets, _) in enumerate(pbar):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)
       
            valid_loss += loss.item()
        
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

    valid_loss /= len(loader)
    valid_acc = accuracy_score(targets_list, preds_list)
    valid_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "epoch": epoch,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc,
        "valid_f1": valid_f1
    }

    return ret


In [66]:
# 학습 및 조기 종료 설정
f1_scores = []
valid_losses = []
best_valid_loss = float('inf')
best_model_path = '/root/best_model.pth'

for epoch in range(EPOCHS):
    print(f"{epoch} epoch")
    trn_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    f1_scores.append(val_ret['valid_f1'])
    valid_losses.append(val_ret['valid_loss'])

    print(f"valid loss : {val_ret['valid_loss']}")
    print(f"valid f1 : {val_ret['valid_f1']}")

    # 최적 모델 저장
    if val_ret['valid_loss'] < best_valid_loss - min_delta:
        best_valid_loss = val_ret['valid_loss']
        torch.save(model.state_dict(), best_model_path)
        print(f"최적 모델이 {best_model_path}에 저장되었습니다.")
        patience_counter = 0  # 성능이 개선되면 카운터 초기화
    else:
        patience_counter += 1

    # 조기 종료 조건 확인
    if patience_counter >= patience:
        print(f"성능 개선이 {patience_counter} 에포크 동안 없었으므로 조기 종료합니다.")
        break

# 최적 모델 로드
best_model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=17).to(device)
best_model.load_state_dict(torch.load(best_model_path, map_location=device))
best_model.eval()
print(f"모델이 {best_model_path}에서 로드되었습니다.")


0 epoch


  0%|          | 0/785 [00:00<?, ?it/s]

Loss: 0.1896:  85%|████████▌ | 669/785 [03:05<00:32,  3.60it/s]

In [ ]:
# 테스트 셋 예측 및 신뢰도 높은 데이터 선별
preds_list = []
confidences_list = []
image_filenames = []

for images, _, filenames in tqdm(tst_loader):  # 파일 이름을 포함하여 3개 값 반환
    images = images.to(device)
    with torch.no_grad():
        preds = best_model(images)
        confidences = torch.softmax(preds, dim=1).max(dim=1)[0]

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    confidences_list.extend(confidences.detach().cpu().numpy())
    image_filenames.extend(filenames)

# 임계값 설정 (예: 0.9 이상)
threshold = 0.9
high_confidence_indices = [i for i, confidence in enumerate(confidences_list) if confidence > threshold]
high_confidence_preds = [preds_list[i] for i in high_confidence_indices]
high_confidence_filenames = [image_filenames[i] for i in high_confidence_indices]

print(high_confidence_filenames)
# 신뢰도 높은 데이터 개수 출력
print(f"신뢰도 {threshold} 이상인 데이터 개수: {len(high_confidence_indices)}")


100%|██████████| 99/99 [00:18<00:00,  5.25it/s]

['0008fdb22ddce0ce.jpg', '00091bffdffd83de.jpg', '00396fbc1f6cc21d.jpg', '00901f504008d884.jpg', '00b33e0ee6d59427.jpg', '00bbdcfbbdb3e131.jpg', '00c03047e0fbef40.jpg', '00c0dabb63ca7a16.jpg', '00dcea90f63ad630.jpg', '00f5784903a39fdd.jpg', '0111a6728e9f8a73.jpg', '0114a887a2c2e4ca.jpg', '016240faa186d24d.jpg', '016b0c00cdf93e0a.jpg', '017ba667291b53c6.jpg', '017e5da799e1637c.jpg', '0182bffa56bdd844.jpg', '019ed42bb4c2caa9.jpg', '01bd84a54be54b8b.jpg', '01c918594307c6f2.jpg', '01ebd05a14e10618.jpg', '021e7c8d9dc19021.jpg', '0298c2151b43d86b.jpg', '02ac23941313841b.jpg', '02b3712bd48b8644.jpg', '02b5e73920c3c54e.jpg', '02f79963274b3c41.jpg', '02ffc27eff468793.jpg', '0366fcb21245a5cc.jpg', '039ff76910d52749.jpg', '03a7dcbe6b74bb8e.jpg', '040ba9bc68f4e380.jpg', '0412f6a5ba912add.jpg', '04284576791e9ec1.jpg', '0447bde0b7da3a6c.jpg', '046c2f2d42bbac35.jpg', '048efa3d18a4999d.jpg', '048fe083d6a62591.jpg', '04917bb8d098898f.jpg', '049445f90a8a2b8d.jpg', '04f502c69bb8dcaa.jpg', '05050540613224

In [ ]:
# DataFrame 생성
additional_df = pd.DataFrame({
    'ID': high_confidence_filenames,
    'target': high_confidence_preds
})

# 새로운 CSV 파일로 저장
additional_df.to_csv('/root/data/high_confidence_data_3.csv', index=False)
print(f"신뢰도 0.9 이상인 데이터가 high_confidence_data_3.csv에 저장되었습니다.")

신뢰도 0.9 이상인 데이터가 high_confidence_data_3.csv에 저장되었습니다.


In [ ]:
# 기존 train.csv 파일 로드
train_df = pd.read_csv('/root/data/train.csv')

# high_confidence_data.csv 파일 로드
high_confidence_df = pd.read_csv('/root/data/high_confidence_data_3.csv')

# 두 데이터프레임 병합
combined_df = pd.concat([train_df, high_confidence_df], ignore_index=True)

# 새로운 CSV 파일로 저장
combined_csv_path = '/root/data/combined_train_data_3.csv'
combined_df.to_csv(combined_csv_path, index=False)
print(f"기존 train.csv와 high_confidence_data.csv가 병합된 데이터가 {combined_csv_path}에 저장되었습니다.")

기존 train.csv와 high_confidence_data.csv가 병합된 데이터가 /root/data/combined_train_data_3.csv에 저장되었습니다.


# TEST

In [ ]:
preds_list = []

best_model.eval()

for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv(f"{RESULT_CSV_PATH}/base.csv", index=False)

100%|██████████| 99/99 [00:17<00:00,  5.59it/s]


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.optim import Adam
import timm
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, Dataset, ConcatDataset

# 모델 생성 및 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=17).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# 한 epoch 학습 함수
def train_one_epoch(loader, model, optimizer, loss_fn, device, epoch):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for step, (image, targets) in enumerate(pbar):
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "model": model,
        "train_epoch": epoch,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1
    }

    return ret

# 한 epoch 검증 함수
def valid_one_epoch(loader, model, loss_fn, device, epoch):
    model.eval()
    valid_loss = 0

    preds_list = []
    targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader)
        for step, (image, targets) in enumerate(pbar):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)
       
            valid_loss += loss.item()
        
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

    valid_loss /= len(loader)
    valid_acc = accuracy_score(targets_list, preds_list)
    valid_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "epoch": epoch,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc,
        "valid_f1": valid_f1
    }

    return ret

# 학습 및 조기 종료 설정
f1_scores = []
valid_losses = []
trained_models = []
patient_counter = 0
best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    print(f"{epoch} epoch")
    trn_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    f1_scores.append(val_ret['valid_f1'])
    valid_losses.append(val_ret['valid_loss'])
    trained_models.append(trn_ret['model'])

    print(f"valid loss : {val_ret['valid_loss']}")
    print(f"valid f1 : {val_ret['valid_f1']}")

    # 성능 개선 됨
    if val_ret['valid_loss'] < best_valid_loss - min_delta:
        best_valid_loss = val_ret['valid_loss']
        patience_counter = 0  
    # 성능 개선 되지 않음
    else:
        patience_counter += 1  

    # 성능 개선이 patience 만큼 안되면 학습 중단
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

best_model_idx = np.argmin(np.array(valid_losses))
best_model = trained_models[best_model_idx]

# 테스트 셋 예측 및 신뢰도 높은 데이터 선별
preds_list = []
confidences_list = []
image_filenames = []

best_model.eval()

for image, filename in tqdm(tst_loader):
    image = image.to(device)
    image_filenames.extend(filename)

    with torch.no_grad():
        preds = best_model(image)
        confidences = torch.softmax(preds, dim=1).max(dim=1)[0]

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    confidences_list.extend(confidences.detach().cpu().numpy())

# 임계값 설정 (예: 0.9 이상)
threshold = 0.9
high_confidence_indices = [i for i, confidence in enumerate(confidences_list) if confidence > threshold]
high_confidence_preds = [preds_list[i] for i in high_confidence_indices]
high_confidence_images = [tst_loader.dataset[i][0] for i in high_confidence_indices]
high_confidence_filenames = [image_filenames[i] for i in high_confidence_indices]

# 신뢰도 높은 데이터 개수 출력
print(f"신뢰도 {threshold} 이상인 데이터 개수: {len(high_confidence_indices)}")



0 epoch


Loss: 0.9199: 100%|██████████| 10/10 [00:01<00:00,  5.95it/s]


valid loss : 0.5735359519720078
valid f1 : 0.8333208447772346
1 epoch


Loss: 0.6984: 100%|██████████| 10/10 [00:01<00:00,  5.97it/s]


valid loss : 0.6423878818750381
valid f1 : 0.8155217741065786
2 epoch


...

valid loss : 0.8146006405353546
valid f1 : 0.8694208654723347
5 epoch


Loss: 1.3031: 100%|██████████| 10/10 [00:01<00:00,  6.01it/s]


valid loss : 0.9452517539262771
valid f1 : 0.846523673530172
Early stopping at epoch 5


100%|██████████| 99/99 [00:18<00:00,  5.46it/s]


신뢰도 0.9 이상인 데이터 개수: 2784


In [ ]:
# 기존 학습 데이터셋
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# 기존 학습 데이터 로더에서 데이터셋 추출
train_images = [item[0] for item in trn_loader.dataset]
train_labels = [item[1] for item in trn_loader.dataset]




NameError: name 'Dataset' is not defined

In [ ]:
# 새로운 학습 데이터 로더 생성
new_trn_loader = DataLoader(new_train_dataset, batch_size=trn_loader.batch_size, shuffle=True)

print(f"새로운 학습 데이터셋 크기: {len(new_train_dataset)}")

# 합친 데이터셋을 새로운 파일로 저장
combined_dataset_path = '/root/combined_dataset.csv'

# 기존 train 데이터 로드
train_df = pd.read_csv('train.csv')

# 추가할 데이터프레임 생성
additional_df = pd.DataFrame({
    'ID': high_confidence_filenames,
    'target': high_confidence_preds
})

# 기존 train 데이터와 추가 데이터 병합
combined_df = pd.concat([train_df, additional_df], ignore_index=True)

# 새로운 CSV 파일로 저장
combined_df.to_csv(combined_dataset_path, index=False)
print(f"합친 데이터셋이 {combined_dataset_path}에 저장되었습니다.")


In [ ]:
# 추가된 데이터로 모델 다시 학습
for epoch in range(EPOCHS):
    print(f"{epoch} epoch (with additional data)")
    trn_ret = train_one_epoch(new_trn_loader, model, optimizer, loss_fn, device, epoch)
    val_ret = valid_one_epoch(val_loader, model, loss_fn, device, epoch)

    f1_scores.append(val_ret['valid_f1'])
    valid_losses.append(val_ret['valid_loss'])
    trained_models.append(trn_ret['model'])

    print(f"valid loss : {val_ret['valid_loss']}")
    print(f"valid f1 : {val_ret['valid_f1']}")

    # 성능 개선 됨
    if val_ret['valid_loss'] < best_valid_loss - min_delta:
        best_valid_loss = val_ret['valid_loss']
        patience_counter = 0  
    # 성능 개선 되지 않음
    else:
        patience_counter += 1  

    # 성능 개선이 patience 만큼 안되면 학습 중단
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

best_model_idx = np.argmin(np.array(valid_losses))
best_model = trained_models[best_model_idx]

In [ ]:
# 최종 예측 결과 저장
preds_list = []

best_model.eval()

for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv(f"{RESULT_CSV_PATH}/base.csv", index=False)


# 결과 분석

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# 예측 결과 생성
valid_preds_list = []

best_model.eval()

for image, _ in tqdm(val_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = best_model(image)
        
    valid_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 실제 레이블과 예측 레이블 준비
true_labels = val_dataset.get_labels()
pred_labels = np.array(valid_preds_list)

# Confusion Matrix 생성
cm = confusion_matrix(true_labels, pred_labels)

# 클래스의 최대값 확인 (히트맵의 크기를 결정하기 위해)
n_classes = max(cm.shape)

# 히트맵 생성
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(n_classes), 
            yticklabels=range(n_classes))

plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# x축과 y축의 눈금을 1단위로 설정
plt.xticks(np.arange(0, n_classes, 1))
plt.yticks(np.arange(0, n_classes, 1))

plt.tight_layout()
plt.show()


100%|██████████| 10/10 [00:01<00:00,  5.90it/s]


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets